<a href="https://colab.research.google.com/github/sergiokapone/DataScience/blob/main/Hw7/Hw7_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання
Візьміть датасет [movielens](https://surprise.readthedocs.io/en/stable/dataset.html) і побудуйте модель матричної факторизації. У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації, також поекспериментуйте з іншими [алгоритмами](https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html) розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.


In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163339 sha256=6772ce975a49d0d0d4afc367c69ac85a5b72f771534205af2c7b3b9232f83fa5
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
from surprise import SVD, SVDpp, NMF
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate
from hyperopt import hp, fmin, tpe


In [3]:
# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [4]:
# sample random trainset and testset
# test set is made of 25% of the ratings.
train_set, test_set = train_test_split(data, test_size=0.25)

In [5]:
models = [SVD, SVDpp, NMF]

In [6]:
space_svd = {
    'n_factors': hp.choice('n_factors', [50, 100, 150]),
    'n_epochs': hp.choice('n_epochs', [10, 20, 30]),
    'lr_all': hp.uniform('lr_all', 0.001, 0.1),
    'reg_all': hp.uniform('reg_all', 0.01, 0.2),
    'random_state': 42
}

space_nmf = {
        'n_factors': hp.choice('n_factors', [5, 10, 15, 20, 25]),
        'n_epochs': hp.choice('n_epochs', [10, 20, 30, 40, 50]),
        'biased': hp.choice('biased', [True, False]),
        'reg_pu': hp.uniform('reg_pu', 0.001, 0.1),
        'reg_qi': hp.uniform('reg_qi', 0.001, 0.1),
        'lr_bu': hp.uniform('lr_bu', 0.001, 0.1),
        'lr_bi': hp.uniform('lr_bi', 0.001, 0.1),
        'random_state': 42
    }

In [9]:
# Визначте функцію для обчислення RMSE
def objective(params, func):
    model = func(**params)
    model.fit(train_set)
    predictions = model.test(test_set)
    rmse = accuracy.rmse(predictions)
    return rmse

best_params = []
for i, func in enumerate(models):
    print('-------------------------')
    print(f'Srart: {func.__name__}')
    print('-------------------------')
    space=[space_svd, space_svd, space_nmf]

    best = fmin(fn=lambda params: objective(params, func), space=space[i], algo=tpe.suggest, max_evals=1)
    best_params.append(best)

best_params

-------------------------
Srart: SVD
-------------------------
RMSE: 0.9495
100%|██████████| 1/1 [00:01<00:00,  1.95s/trial, best loss: 0.949518700420323]
-------------------------
Srart: SVDpp
-------------------------
RMSE: 0.9944
100%|██████████| 1/1 [00:31<00:00, 31.63s/trial, best loss: 0.994387259694318]
-------------------------
Srart: NMF
-------------------------
RMSE: 2.2305
100%|██████████| 1/1 [00:00<00:00,  1.79trial/s, best loss: 2.2305246553620024]


[{'lr_all': 0.09183278659581284,
  'n_epochs': 1,
  'n_factors': 2,
  'reg_all': 0.1014529512908898},
 {'lr_all': 0.07478450453466987,
  'n_epochs': 0,
  'n_factors': 0,
  'reg_all': 0.04280976199614765},
 {'biased': 1,
  'lr_bi': 0.03444007874332134,
  'lr_bu': 0.04188214291306692,
  'n_epochs': 0,
  'n_factors': 0,
  'reg_pu': 0.0030900681491158634,
  'reg_qi': 0.027713820683042168}]

In [19]:
for best_param, Model in zip(best_params, models):
    print('-------------------------')
    print(f'Srart: {Model.__name__}')
    print('-------------------------\n')
    model = Model(**best_param)
    results = cross_validate(model, data, measures=['MSE', 'MAE'], cv=5, verbose=True)
    print('\n')

-------------------------
Srart: SVD
-------------------------

Evaluating MSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MSE (testset)     0.9528  0.9681  0.9768  0.9572  0.9616  0.9633  0.0084  
MAE (testset)     0.7803  0.7831  0.7843  0.7783  0.7777  0.7808  0.0026  
Fit time          0.06    0.08    0.08    0.08    0.09    0.08    0.01    
Test time         0.14    0.25    0.11    0.25    0.11    0.17    0.06    


-------------------------
Srart: SVDpp
-------------------------

Evaluating MSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MSE (testset)     1.2672  1.2758  1.2640  1.2763  1.2524  1.2671  0.0088  
MAE (testset)     0.9440  0.9473  0.9459  0.9471  0.9393  0.9447  0.0030  
Fit time          0.06    0.06    0.07    0.06    0.07    0.06    0.00    
Test time         4.14    3.00    2.92    3.03    4.19    3.46    0.58    


--------

# Висновки

На основі результатів крос-валідації для різних алгоритмів матричної факторизації (SVD, SVD++, NMF), можна зробити такі висновки:

1. SVD має середнє значення MSE приблизно 0.9633 та MAE приблизно 0.7808. Час навчання та тестування відносно низький.

2. SVD++ має значення MSE приблизно 1.2671 та MAE приблизно 0.9447. Час тестування відносно високий, що може бути недоцільним у великих завданнях.

3. NMF має значення MSE приблизно 1.2672 та MAE приблизно 0.9447. Час навчання та тестування прийнятний, але середня точність менше, ніж у SVD.

За зазначеними метриками RMSE і MAE, SVD має найкращу точність серед цих алгоритмів. Таким чином, для цього конкретного завдання рекомендується використовувати алгоритм SVD для матричної факторизації.